In [2]:
import os
os.chdir('../')

from ast import arg
from DeepMTP.main import DeepMTP
from DeepMTP.hpo_worker import BaseWorker
from DeepMTP.random_search import RandomSearch
from DeepMTP.dataset import load_process_MLC, load_process_MTR, load_process_DP, process_dummy_MLC, process_dummy_MTR, process_dummy_DP, load_process_MC
from DeepMTP.utils.data_utils import data_process, BaseDataset
from DeepMTP.utils.utils import generate_config
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

In [3]:
# define the configuration space
cs= CS.ConfigurationSpace()

lr= CSH.UniformFloatHyperparameter(
    "learning_rate", lower=1e-6, upper=1e-3, default_value="1e-3", log=True
)
cs.add_hyperparameters([lr])

embedding_size= CSH.UniformIntegerHyperparameter(
    "embedding_size", lower=8, upper=2048, default_value=64, log=False
)

instance_branch_layers= CSH.UniformIntegerHyperparameter(
    "instance_branch_layers", lower=1, upper=2, default_value=1, log=False
)

instance_branch_nodes_per_layer= CSH.UniformIntegerHyperparameter(
    "instance_branch_nodes_per_layer", lower=8, upper=2048, default_value=64, log=False
)

target_branch_layers = CSH.UniformIntegerHyperparameter(
    "target_branch_layers", lower=1, upper=2, default_value=1, log=False
)

target_branch_nodes_per_layer = CSH.UniformIntegerHyperparameter(
    "target_branch_nodes_per_layer", lower=8, upper=2048, default_value=64, log=False
)

dropout_rate = CSH.UniformFloatHyperparameter(
    "dropout_rate", lower=0.0, upper=0.9, default_value=0.4, log=False
)

batch_norm = CSH.CategoricalHyperparameter("batch_norm", [True, False])

cs.add_hyperparameters(
    [
        embedding_size,
        instance_branch_layers,
        instance_branch_nodes_per_layer,
        target_branch_layers,
        target_branch_nodes_per_layer,
        dropout_rate,
        batch_norm,
    ]
)

cond = CS.GreaterThanCondition(dropout_rate, instance_branch_layers, 1)
cond2 = CS.GreaterThanCondition(batch_norm, instance_branch_layers, 1)
cond3 = CS.GreaterThanCondition(dropout_rate, target_branch_layers, 1)
cond4 = CS.GreaterThanCondition(batch_norm, target_branch_layers, 1)

cs.add_condition(CS.OrConjunction(cond, cond3))
cs.add_condition(CS.OrConjunction(cond2, cond4))

(batch_norm | instance_branch_layers > 1 || batch_norm | target_branch_layers > 1)

In [4]:
data = load_process_MLC(dataset_name='yeast', variant='undivided')
train, val, test, data_info = data_process(data, validation_setting='B', verbose=True)

Processing...
yeast:undivided - exists, not redownloading
Done
Interaction file: 2d numpy array format detected
Interaction file: checking format consistency... Passed
Interaction file: checking instance id format consistency... Passed
Interaction file: checking target id type consistency... Passed

Interaction file: checking target variable type consistency... Passed
Automatically detected type of target variable type: binary

-- Test set was not provided, could not detect if novel instances exist or not 
-- Test set was not provided, could not detect if novel targets exist or not 

Instance features file: processing features... Done

Cross input consistency for (numpy) interaction data and instance features checks out
-- Same instance ids in the interaction and features files for the train set

Splitting train to train-test according to validation setting B... Done
Splitting train to train-val according to validation setting B... Done


In [5]:
data_info

{'detected_validation_setting': 'B',
 'detected_problem_mode': 'classification',
 'instance_branch_input_dim': 103,
 'target_branch_input_dim': 14}

In [6]:
config = {    

    'hpo_results_path': './random_search/',

    'instance_branch_input_dim': data_info['instance_branch_input_dim'],
    'target_branch_input_dim': data_info['target_branch_input_dim'],
    'validation_setting': data_info['detected_validation_setting'],
    'general_architecture_version': 'dot_product',
    'problem_mode': data_info['detected_problem_mode'],

    'compute_mode': 'cuda:1',
    'train_batchsize': 512,
    'val_batchsize': 512,
    'num_epochs': 10,
    'num_workers': 8,

    'metrics': ['hamming_loss', 'auroc'],
    'metrics_average': ['macro'],
    'patience': 10,

    'evaluate_train': True,
    'evaluate_val': True,

    'verbose': True,
    'results_verbose': False,
    'use_early_stopping': True,
    'use_tensorboard_logger': True,
    'wandb_project_name': None,
    'wandb_project_entity': None,
    'metric_to_optimize_early_stopping': 'loss',
    'metric_to_optimize_best_epoch_selection': 'loss',

    'instance_branch_architecture': 'MLP',

    'target_branch_architecture': 'MLP',

    'save_model': True,

    'eval_every_n_epochs': 10,

    'running_hpo': True,

    'additional_info': {'budget': 9}
    }

In [7]:
worker = BaseWorker(
    train, val, test, data_info, config, 'loss'
)

In [8]:
worker.base_config

{'hpo_results_path': './random_search/',
 'instance_branch_input_dim': 103,
 'target_branch_input_dim': 14,
 'validation_setting': 'B',
 'enable_dot_product_version': True,
 'problem_mode': 'classification',
 'compute_mode': 'cuda:1',
 'train_batchsize': 512,
 'val_batchsize': 512,
 'num_epochs': 10,
 'num_workers': 8,
 'metrics': ['hamming_loss', 'auroc'],
 'metrics_average': ['macro'],
 'patience': 10,
 'evaluate_train': True,
 'evaluate_val': True,
 'verbose': True,
 'results_verbose': False,
 'use_early_stopping': True,
 'use_tensorboard_logger': True,
 'wandb_project_name': 'DeepMTP_v2',
 'wandb_project_entity': 'diliadis',
 'metric_to_optimize_early_stopping': 'loss',
 'metric_to_optimize_best_epoch_selection': 'loss',
 'instance_branch_architecture': 'MLP',
 'target_branch_architecture': 'MLP',
 'save_model': True,
 'eval_every_n_epochs': 10,
 'running_hpo': True,
 'additional_info': {'budget': 9}}

In [9]:
hb = RandomSearch(
    base_worker=worker,
    configspace=cs,
    budget=config['additional_info']['budget'],
    max_num_epochs=config['num_epochs'], 
    direction="min",
    verbose=True
)

In [10]:
best_overall_config = hb.run_optimizer()

---- Evaluating configuration... 
Selected device: cuda:1
Starting training...


2022-06-20 15:27:31.164 ERROR   wandb.jupyter: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: diliadis. Use `wandb login --relogin` to force relogin


Epoch:0... Done
  Validating... Calculating val performance... Done
Done
Epoch:1... Done
  Validating... Done
Epoch:2... Done
  Validating... Done
Epoch:3... Done
  Validating... Done
Epoch:4... Done
  Validating... Done
Epoch:5... Done
  Validating... Done
Epoch:6... Done
  Validating... Done
Epoch:7... Done
  Validating... Done
Epoch:8... Done
  Validating... Done
Epoch:9... Done
  Validating... Calculating val performance... Done
Done
Starting testing... Calculating test performance... Done
Done


test_auroc_macro,▁
test_hamming_loss_macro,▁
train_auroc_macro,▁█
train_hamming_loss_macro,█▁
train_loss,█▄▄▄▄▄▄▂▂▁
val_auroc_macro,▁█
val_hamming_loss_macro,█▁
val_loss,█████▇▅▃▂▁
test_auroc_macro,0.54507
test_hamming_loss_macro,0.27214
train_auroc_macro,0.52284


+-------+--------+--------+--------------------+-------------+
|  mode | #epoch |  loss  | hamming_loss_macro | auroc_macro |
+-------+--------+--------+--------------------+-------------+
| train |   0    | 0.8865 |       0.4225       |    0.5023   |
| train |   1    | 0.6928 |         -          |      -      |
| train |   2    | 0.6925 |         -          |      -      |
| train |   3    | 0.692  |         -          |      -      |
| train |   4    | 0.6907 |         -          |      -      |
| train |   5    | 0.6856 |         -          |      -      |
| train |   6    | 0.6612 |         -          |      -      |
| train |   7    | 0.605  |         -          |      -      |
| train |   8    | 0.5682 |         -          |      -      |
| train |   9    | 0.531  |       0.3028       |    0.5228   |
+-------+--------+--------+--------------------+-------------+
+------+--------+--------+--------------------+-------------+----------------+
| mode | #epoch |  loss  | hamming_loss

...

test_auroc_macro,▁
test_hamming_loss_macro,▁
train_auroc_macro,▁█
train_hamming_loss_macro,█▁
train_loss,█▆▂▂▁▁▁▁▁▁
val_auroc_macro,▁█
val_hamming_loss_macro,█▁
val_loss,█▄▂▁▁▁▁▁▁▁
test_auroc_macro,0.53204
test_hamming_loss_macro,0.69599
train_auroc_macro,0.52838


+-------+--------+---------+--------------------+-------------+
|  mode | #epoch |   loss  | hamming_loss_macro | auroc_macro |
+-------+--------+---------+--------------------+-------------+
| train |   0    | 69.7071 |       0.6972       |    0.4943   |
| train |   1    | 48.3088 |         -          |      -      |
| train |   2    | 14.6907 |         -          |      -      |
| train |   3    |  6.6439 |         -          |      -      |
| train |   4    |  5.0357 |         -          |      -      |
| train |   5    |  3.9442 |         -          |      -      |
| train |   6    |  3.1283 |         -          |      -      |
| train |   7    |  2.4999 |         -          |      -      |
| train |   8    |  2.016  |         -          |      -      |
| train |   9    |  1.6489 |       0.6956       |    0.5284   |
+-------+--------+---------+--------------------+-------------+
+------+--------+---------+--------------------+-------------+----------------+
| mode | #epoch |   loss

Epoch:0... Done
  Validating... Calculating val performance... Done
Done
Epoch:1... Done
  Validating... Done
Epoch:2... Done
  Validating... Done
Epoch:3... Done
  Validating... Done
Epoch:4... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 3
Epoch:5... Done
  Validating... Done
-----------------------------EarlyStopping counter: 2 out of 10---------------------- best epoch currently 3
Epoch:6... Done
  Validating... Done
-----------------------------EarlyStopping counter: 3 out of 10---------------------- best epoch currently 3
Epoch:7... Done
  Validating... Done
-----------------------------EarlyStopping counter: 4 out of 10---------------------- best epoch currently 3
Epoch:8... Done
  Validating... Done
-----------------------------EarlyStopping counter: 5 out of 10---------------------- best epoch currently 3
Epoch:9... Done
  Validating... Calculating val performance... Done
Done
-------------

test_auroc_macro,▁
test_hamming_loss_macro,▁
train_auroc_macro,▁█
train_hamming_loss_macro,█▁
train_loss,█▄▁▁▁▁▁▁▁▁
val_auroc_macro,▁█
val_hamming_loss_macro,█▁
val_loss,█▁▁▁▁▁▁▁▁▁
test_auroc_macro,0.68014
test_hamming_loss_macro,0.20484
train_auroc_macro,0.93145


+-------+--------+---------+--------------------+-------------+
|  mode | #epoch |   loss  | hamming_loss_macro | auroc_macro |
+-------+--------+---------+--------------------+-------------+
| train |   0    |  69.674 |       0.6972       |    0.5095   |
| train |   1    | 25.7964 |         -          |      -      |
| train |   2    |  0.462  |         -          |      -      |
| train |   3    |  0.4177 |         -          |      -      |
| train |   4    |  0.3791 |         -          |      -      |
| train |   5    |  0.3396 |         -          |      -      |
| train |   6    |  0.2914 |         -          |      -      |
| train |   7    |  0.2641 |         -          |      -      |
| train |   8    |  0.2299 |         -          |      -      |
| train |   9    |  0.2035 |       0.0747       |    0.9315   |
+-------+--------+---------+--------------------+-------------+
+------+--------+---------+--------------------+-------------+----------------+
| mode | #epoch |   loss

# Load the best model and generate predictions for the test set

In [ ]:
best_model = DeepMTP(best_overall_config.info['config'], checkpoint_dir=best_overall_config.info['model_dir'])

best_model = DeepMTP(best_overall_config.info['config'], best_overall_config.info['model_dir'])

In [ ]:
best_model_results = best_model.predict(test, verbose=True)

In [ ]:
best_model_results